<a href="https://colab.research.google.com/github/saicharan-r/ERAv2/blob/main/S6/Session_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.conv1 = nn.Conv2d(1, 10, 3) #input -28 OUtput 26 RF
        # self.conv2 = nn.Conv2d(10, 32, 3) #input -26 OUtput 24 RF
        # self.pool = nn.MaxPool2d(2,2)
        # self.conv3= nn.Conv2d(in_channels=32,out_channels=10,kernel_size=1)
        # self.conv4= nn.Conv2d(in_channels=10,out_channels=16,kernel_size=3)
        # self.conv5= nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3)

        # self.conv6= nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3)
        # self.conv7= nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3)
        # self.conv8 = nn.Conv2d(in_channels=32, out_channels=10, kernel_size=1)

        self.layer1=nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3), #16,26,26
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(16),
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3), #16,24,24
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2,2) #16,12,12
        )
        self.layer2=nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3), #16,10,10
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(16),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3), #16,8,8
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2) #16,4,4
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3), #32,2,2
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 10, 1), # 10,2,2
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(10),
        )
        self.gaplayer = nn.Sequential(
            nn.AvgPool2d(2)
        )




    def forward(self, x):
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.gaplayer(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [26]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
           Dropout-3           [-1, 16, 26, 26]               0
       BatchNorm2d-4           [-1, 16, 26, 26]              32
            Conv2d-5           [-1, 16, 24, 24]           2,320
              ReLU-6           [-1, 16, 24, 24]               0
           Dropout-7           [-1, 16, 24, 24]               0
       BatchNorm2d-8           [-1, 16, 24, 24]              32
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10           [-1, 16, 10, 10]           2,320
             ReLU-11           [-1, 16, 10, 10]               0
          Dropout-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
           Conv2d-14             [-1, 3

<ipython-input-25-dde2677ad44e>:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [27]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:04<00:00, 2138166.29it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 485862.38it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4385832.07it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4368385.41it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [28]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [29]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-25-dde2677ad44e>:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.21719926595687866 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 19.65it/s]



Test set: Average loss: 0.0995, Accuracy: 9841/10000 (98%)



loss=0.07014629244804382 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.45it/s]



Test set: Average loss: 0.0518, Accuracy: 9887/10000 (99%)



loss=0.03289469704031944 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.23it/s]


...


Test set: Average loss: 0.0239, Accuracy: 9931/10000 (99%)



loss=0.03599603846669197 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.06it/s]



Test set: Average loss: 0.0279, Accuracy: 9929/10000 (99%)



loss=0.003205817425623536 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.49it/s]



Test set: Average loss: 0.0228, Accuracy: 9933/10000 (99%)

